In [0]:
LANDING_PATH = "abfss://landing-dev@stcryptomedallion.dfs.core.windows.net/"
BRONZE_PATH = "abfss://bronze-dev@stcryptomedallion.dfs.core.windows.net/"


In [0]:
# Autoloader for Binance
df_binance = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.schemaLocation", f"{BRONZE_PATH}binance/_schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("cloudFiles.rescuedDataColumn", "_rescued_data")
        .option("recursiveFileLookup", "true")
        .option("multiLine", "true")
        .load(f"{LANDING_PATH}binance/")
)

(
    df_binance.writeStream
        .format("delta")
        .option("checkpointLocation", f"{BRONZE_PATH}binance/_checkpoint")
        .outputMode("append")
        .trigger(availableNow=True)
        .toTable("crypto.bronze.binance_raw")
)

In [0]:
#Autoloader for Bybit
df_bybit = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option("cloudFiles.schemaLocation", f"{BRONZE_PATH}bybit/_schema")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("cloudFiles.rescuedDataColumn", "_rescued_data")
        .option("recursiveFileLookup", "true")
        .option("multiLine", "true")
        .load(f"{LANDING_PATH}bybit/")
)

(
    df_bybit.writeStream
        .format("delta")
        .option("checkpointLocation", f"{BRONZE_PATH}bybit/_checkpoint")
        .outputMode("append")
        .trigger(availableNow=True)
        .toTable("crypto.bronze.bybit_raw")
)